In [6]:
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta

In [7]:
schedule_paths = [
    'collected_schedules/best_schedule_bus600_gen1.csv',
    'collected_schedules/best_schedule_bus600_gen2.csv',
    'collected_schedules/best_schedule_bus600_gen3.csv',
    'collected_schedules/best_schedule_bus600_gen4.csv',
    'collected_schedules/best_schedule_bus600_gen5.csv',
    'collected_schedules/best_schedule_bus600_gen6.csv',
    'collected_schedules/best_schedule_bus600_gen7.csv',
    'collected_schedules/best_schedule_bus600_gen8.csv',
    'collected_schedules/best_schedule_bus600_gen9.csv',
    'collected_schedules/best_schedule_bus600_gen10.csv',
    'collected_schedules/best_schedule_bus600_gen11.csv',
    'collected_schedules/best_schedule_bus600_gen12.csv',
    'collected_schedules/best_schedule_bus600_gen13.csv',
    'collected_schedules/best_schedule_bus600_gen14.csv',
    'collected_schedules/best_schedule_bus600_gen15.csv',
]

schedule1 = pd.read_csv(schedule_paths[0]).rename(columns={'Bus': 'bus', 'Trip ID': 'trip_id'})
schedule2 = pd.read_csv(schedule_paths[1]).rename(columns={'Bus': 'bus', 'Trip ID': 'trip_id'})
schedule3 = pd.read_csv(schedule_paths[2]).rename(columns={'Bus': 'bus', 'Trip ID': 'trip_id'})
schedule4 = pd.read_csv(schedule_paths[3]).rename(columns={'Bus': 'bus', 'Trip ID': 'trip_id'})
schedule5 = pd.read_csv(schedule_paths[4]).rename(columns={'Bus': 'bus', 'Trip ID': 'trip_id'})
schedule6 = pd.read_csv(schedule_paths[5]).rename(columns={'Bus': 'bus', 'Trip ID': 'trip_id'})
schedule7 = pd.read_csv(schedule_paths[6]).rename(columns={'Bus': 'bus', 'Trip ID': 'trip_id'})
schedule8 = pd.read_csv(schedule_paths[7]).rename(columns={'Bus': 'bus', 'Trip ID': 'trip_id'})
schedule9 = pd.read_csv(schedule_paths[8]).rename(columns={'Bus': 'bus', 'Trip ID': 'trip_id'})

In [8]:
schedules = [schedule1, schedule2, schedule3, schedule4, schedule5, schedule6, schedule7, schedule8, schedule9]

In [9]:
trips = pd.read_csv('formatted_DMRC_trips.csv')
stops = pd.read_csv('DMRC_Data/stops.txt')
times = pd.read_csv('DMRC_Data/stop_times.txt')

print(trips.head())
print(stops.head())
print(times.head())

   Unnamed: 0  route_id service_id  trip_id  trip_headsign  trip_short_name  \
0           0         0    weekday        0            NaN              NaN   
1           1         0    weekday        1            NaN              NaN   
2           2         0    weekday       10            NaN              NaN   
3           3         0    weekday      100            NaN              NaN   
4           4         2    weekday     1000            NaN              NaN   

   direction_id  block_id  shape_id  wheelchair_accessible  bikes_allowed  \
0           NaN       NaN  shp_1_30                      0              0   
1           NaN       NaN  shp_1_30                      0              0   
2           NaN       NaN  shp_1_30                      0              0   
3           NaN       NaN  shp_1_30                      0              0   
4           NaN       NaN  shp_1_13                      0              0   

  start_time_y end_time_y  start_stop_y  end_stop_y  
0     05

In [10]:
# Clean and merge stops and times
stops_cleaned = stops.drop(columns=['stop_code', 'stop_desc'])
times_cleaned = times.drop(columns=['stop_headsign', 'pickup_type', 'drop_off_type', 'timepoint', 'continuous_pickup', 'continuous_drop_off'])
stops_times_merge = pd.merge(times_cleaned, stops_cleaned)

In [11]:
def process_bus_schedule(schedule, trips, stops_times_merge):
    
    # Merge schedule with trips
    schedule_trips_merge = pd.merge(schedule, trips, on='trip_id').drop(
        columns=['service_id', 'trip_headsign', 'trip_short_name', 'direction_id',
                    'block_id', 'wheelchair_accessible', 'bikes_allowed']
    )

    # Calculate trip distances
    trip_distances = stops_times_merge.groupby('trip_id')['shape_dist_traveled'].last() / 1000
    schedule_combined = pd.merge(schedule_trips_merge, trip_distances, on='trip_id')

    # Prepare bus statistics
    bus_statistics = pd.DataFrame(schedule_combined.groupby('bus')['shape_dist_traveled'].agg('sum'))
    bus_statistics['trip_count'] = schedule_combined.groupby('bus').size()

    # Adjust time function
    def adjust_time_to_next_day(time_str):
        try:
            time_parts = time_str.split(':')
            hour = int(time_parts[0])
            if hour >= 24:
                adjusted_hour = hour - 24
                adjusted_time_str = f"{adjusted_hour:02}:{time_parts[1]}:{time_parts[2]}"
                return pd.to_datetime(adjusted_time_str) + pd.Timedelta(days=1)
            else:
                return pd.to_datetime(time_str)
        except Exception:
            return None

    # Calculate travel time
    schedule_combined_sorted = schedule_combined.sort_values(by=['bus', 'start_time_y'])
    bus_statistics['travel_time'] = (
        pd.to_datetime(schedule_combined_sorted.groupby('bus')['end_time_y'].last().apply(adjust_time_to_next_day)) -
        pd.to_datetime(schedule_combined_sorted.groupby('bus')['start_time_y'].first().apply(adjust_time_to_next_day))
    ).dt.total_seconds() / 3600

    # Additional statistics
    mileage = 3.7
    petrol_price = 94.77
    co2_emission_factor = 2.31
    bus_statistics['petrol_charges'] = bus_statistics['shape_dist_traveled'] / mileage * petrol_price
    bus_statistics['carbon_emissions'] = bus_statistics['shape_dist_traveled'] * mileage * co2_emission_factor

    # Fix time format
    def fix_time_format(time_str):
        try:
            hours, minutes, seconds = map(int, time_str.split(":"))
            if hours >= 24:
                hours -= 24
            return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        except ValueError:
            return time_str

    # Calculate total time and distances
    schedule_combined['start_time_y'] = schedule_combined['start_time_y'].apply(fix_time_format)
    schedule_combined['end_time_y'] = schedule_combined['end_time_y'].apply(fix_time_format)

    schedule_combined = schedule_combined.merge(stops.drop(columns=['stop_code', 'stop_desc']), left_on='start_stop_y', right_on='stop_id')
    schedule_combined = schedule_combined.rename(
        columns={
            'stop_name': 'start_stop_name',
            'stop_lat': 'start_stop_lat',
            'stop_lon': 'start_stop_lon'
        }
    ).drop(columns='stop_id')
    schedule_combined = schedule_combined.merge(stops.drop(columns=['stop_code','stop_desc']),left_on='end_stop_y',right_on='stop_id')
    schedule_combined = schedule_combined.rename(
        columns={
            'stop_name': 'end_stop_name',
            'stop_lat': 'end_stop_lat',
            'stop_lon': 'end_stop_lon',
            'shape_dist_traveled': 'total_distance',
            'start_time_y' :'start_time',
            'end_time_y' :'end_time',
            'start_stop_y' :'start_stop',
            'end_stop_y' :'end_stop',
        }
    ).drop(columns='stop_id')
    new_order = ['bus', 'trip_id', 'route_id', 'shape_id', 'start_time', 
        'start_stop', 'start_stop_name',
        'start_stop_lat', 'start_stop_lon', 'end_time','end_stop', 'end_stop_name', 'end_stop_lat',
        'end_stop_lon', 'total_distance']
    schedule_combined = schedule_combined[new_order]
    # print(schedule_combined.head())

    def calculate_time_gap(start_time_str, end_time_str):
        time_format = "%H:%M:%S"
        start_time = datetime.strptime(start_time_str, time_format)
        end_time = datetime.strptime(end_time_str, time_format)
        if end_time < start_time:
            end_time += timedelta(days=1)
        return end_time - start_time

    schedule_combined['total_time'] = schedule_combined.apply(
        lambda row: calculate_time_gap(row['start_time'], row['end_time']).total_seconds() / 60, axis=1
    )

    # Haversine function for distance calculation
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371.0
        lat1_rad = math.radians(lat1)
        lon1_rad = math.radians(lon1)
        lat2_rad = math.radians(lat2)
        lon2_rad = math.radians(lon2)
        dlat = lat2_rad - lat1_rad
        dlon = lon2_rad - lon1_rad
        a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c

    # Calculate travel distance to the next stop
    schedule_combined = schedule_combined.sort_values(by=['bus', 'start_time'])
    schedule_combined['travel_distance_to_next'] = np.zeros(len(schedule_combined))
    for i in range(len(schedule_combined) - 1):
        if schedule_combined.iloc[i]['bus'] == schedule_combined.iloc[i + 1]['bus']:
            schedule_combined.at[i, 'travel_distance_to_next'] = haversine(
                schedule_combined.iloc[i + 1]['end_stop_lat'],
                schedule_combined.iloc[i + 1]['end_stop_lon'],
                schedule_combined.iloc[i]['start_stop_lat'],
                schedule_combined.iloc[i]['start_stop_lon']
            )

    # Update statistics
    bus_statistics['active_distance'] = schedule_combined.groupby('bus')['total_distance'].agg('sum')
    bus_statistics['active_time'] = schedule_combined.groupby('bus')['total_time'].agg('sum')
    bus_statistics['passive_distance'] = schedule_combined.groupby('bus')['travel_distance_to_next'].agg('sum')
    bus_statistics['activity_ratio'] = bus_statistics['active_distance'] / bus_statistics['passive_distance']
    
    return schedule_combined, bus_statistics


In [12]:
statistics_list = []
# Process each schedule and collect statistics
for i, schedule in enumerate(schedules, start=0):
    _, stats = process_bus_schedule(schedule, trips, stops_times_merge)
    stats['schedule_id'] = f'schedule{i}'  # Add an identifier for each schedule
    statistics_list.append(stats)

In [13]:
statistics_list[8]['trip_count'].min()

np.int64(1)

In [14]:
statistics_list[8]['trip_count'].mean()

np.float64(9.128547579298832)

In [15]:
statistics_list[8]['trip_count'].max()

np.int64(17)

In [16]:
trips.head()

,Unnamed: 0,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,start_time_y,end_time_y,start_stop_y,end_stop_y
0,0,0,weekday,0,NaN,NaN,NaN,NaN,shp_1_30,0,0,05:28:08,06:18:38,21,1
1,1,0,weekday,1,NaN,NaN,NaN,NaN,shp_1_30,0,0,05:35:18,06:25:48,21,1
2,2,0,weekday,10,NaN,NaN,NaN,NaN,shp_1_30,0,0,06:39:48,07:30:18,21,1
3,3,0,weekday,100,NaN,NaN,NaN,NaN,shp_1_30,0,0,16:48:04,17:38:34,21,1
4,4,2,weekday,1000,NaN,NaN,NaN,NaN,shp_1_13,0,0,18:26:21,20:00:41,71,36


In [21]:
trips['route_id'].unique()

array([ 0,  2, 19, 20, 21, 22, 23, 24, 25, 26, 27,  3, 28, 29, 30, 31, 32,
       33, 34, 35,  4,  5,  6,  7,  1,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [24]:
len(trips)

5438

In [23]:
len(times['trip_id'].unique())

5438